In [2]:
# ################## DELETE SESSION ##################
import fiftyone as fo
dataset = fo.load_dataset("Test")
dataset.delete()
# ####################################################

In [3]:
# ################## LIST EXISTING SESSIONS ##################
import fiftyone as fo

datasets = fo.list_datasets()
# dataset = fo.load_dataset("images")
print(fo.list_datasets())

[]


In [4]:
# ################## CREATE NEW SESSION ##################

import fiftyone as fo
import os

name = "18july"

dataset_dir = "Path_To_Your_Dataset"
ds_17jul = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.FiftyOneDataset,
    name=name,
    progress=True
)

session = fo.launch_app(ds_17jul)

ds_17jul.persistent = True

Importing samples...
 100% |█████████████| 16630/16630 [352.0ms elapsed, 0s remaining, 47.8K samples/s]  


In [1]:
# ################## LOAD CURRENT DATASET VIEW ##################
import fiftyone as fo 

ds_17jul = fo.load_dataset('18july')

ds_17jul_session = fo.launch_app(ds_17jul)
ds_17jul_session.wait()

Notebook sessions cannot wait


In [23]:
# ################## EXPORT IMAGES ##################
export_ids = ds_17jul_session.view

export_ids.export(
    dataset_type=fo.types.FiftyOneDataset,
    export_dir="Path_To_Your_Dataset",
    abs_paths=True,
)

Ignoring unsupported parameter 'abs_paths'
Exporting samples...
 100% |████████████████| 16630/16630 [1.8h elapsed, 0s remaining, 1.9 docs/s]      


In [24]:
# ################## RESIZE IMAGES ##################
import os
from PIL import Image

images_path = r"Path_To_Your_Dataset"

heights = []
widths = []

for filename in os.listdir(images_path):
    if filename.lower().endswith(".jpg"):
        file_path = os.path.join(images_path, filename)
        
        try:
            with Image.open(file_path) as image:
                width, height = image.size
                
                if height >= 1:
                    # Set desired dimensions
                    new_height = 640
                    new_width = 640
                    # Option 1: Maintain aspect ratio
                    #a#spect_ratio = width / height
                    #new_width = int(new_height * aspect_ratio)
                    
                    # Option 2: Set width to a specific value
                    # new_width = 6300
                    # new_height = int(new_width / aspect_ratio)
                    
                    # Resize the image
                    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
                    
                    # Save the resized image
                    resized_image.save(file_path)
                    
                    # Update width and height
                    width, height = resized_image.size
                
                # Append the dimensions to the lists
                heights.append(height)
                widths.append(width)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

if heights:
    print("Minimum height:", min(heights))
else:
    print("Error: No images processed.")

Minimum height: 640


In [16]:
# Append Label to File Name
import os
# Rename files to include labels
export_ids = ds_17jul_session.view

# Mark as errors
for sample in export_ids:
    original_filename = sample.filepath.split(os.path.sep)[-1]  # Get the original filename
    label = sample.ground_truth['label']  # Get the label (adjust if your label field is different)
    new_filename = f"{label}_{original_filename}"  # Create new filename
    old_filepath = f"Path_To_Your_Dataset\\{original_filename}"
    new_filepath = f"Path_To_Your_Dataset\\{new_filename}"

    # Rename the file
    if os.path.exists(old_filepath):
        os.rename(old_filepath, new_filepath)

In [1]:
# Move Matching Datasets to Another Directory

import os
import json
import shutil

# Example usage
json_file_path = "Path_To_JSON_File"  # Path to your JSON file
images_directory = "Path_To_Your_Dataset"  # Directory containing images
output_directory = "Path_To_Your_Dataset"  # Directory to move matching files

def move_files_from_json(json_file, images_path, output_directory):
    """
    Read names from a JSON file, find matching files in the images_path, and move them to the output directory.

    :param json_file: Path to the JSON file containing names to search for.
    :param images_path: Path to the directory containing images.
    :param output_directory: Path to the directory where matching files should be moved.
    """
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Load names from the JSON file
    try:
        with open(json_file, 'r') as file:
            data = json.load(file)
    except Exception as e:
        print(f"Error reading JSON file: {e}")
        return

    # Extract names from JSON
    if isinstance(data, list):
        # Assume JSON contains a list of strings or objects with a 'name' key
        names = [item if isinstance(item, str) else item.get('name') for item in data]
    else:
        print("Invalid JSON format. Expected a list of names or objects with 'name' keys.")
        return

    # Ensure names are valid strings
    names = [name for name in names if isinstance(name, str)]

    # Iterate through files in the images_path
    for file_name in os.listdir(images_path):
        # Check if the file name matches any of the names from the JSON (without extensions)
        name_without_extension, _ = os.path.splitext(file_name)
        if name_without_extension in names:
            source_path = os.path.join(images_path, file_name)
            destination_path = os.path.join(output_directory, file_name)
            try:
                shutil.move(source_path, destination_path)
                print(f"Moved: {file_name} to {output_directory}")
            except Exception as e:
                print(f"Error moving file {file_name}: {e}")

move_files_from_json(json_file_path, images_directory, output_directory)
